In [ ]:
#install what needs to be installed setting up intance here
!pip install -U bitsandbytes
!pip install accelerate

In [ ]:
#Google Drive mount here
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
#enabling import of custom module for colab
! cp /content/gdrive/MyDrive/LLMHallucinations/Util.py .

In [1]:
# import libraries
from Util import load_data_JSON, load_model, write_out
import pandas as pd
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch

ModuleNotFoundError: No module named 'huggingface_hub'

In [ ]:
#load the data
general_df,prompt_g,correct_g,gtp4_g,Info_g = load_data_JSON('/content/gdrive/MyDrive/LLMHallucinations/Input files/general_data.json','general')
qa_df,prompt_q,correct_q,gtp4_q,Info_q = load_data_JSON('/content/gdrive/MyDrive/LLMHallucinations/Input files/qa_data.json','qa')
sum_df,prompt_s,correct_s,gtp4_s,Info_s = load_data_JSON('/content/gdrive/MyDrive/LLMHallucinations/Input files/summarization_data.json','sum')

In [ ]:
access = input('Access code?')

In [ ]:
#setup of model
tokenizer, model = load_model("meta-llama/Llama-2-7b-chat-hf",access)

In [ ]:
gen = pipeline("text-generation", model=model,torch_dtype = torch.float16, tokenizer=tokenizer)

In [ ]:
def get_response(prompt,max_len):
  sequences = gen(prompt,do_sample=True,top_k = 5, num_return_sequences =1, eos_token_id = tokenizer.eos_token_id, max_length = max_len +len(prompt) )
  return (sequences[0]['generated_text'])

In [ ]:
#run of the prompts 
out_g =[]
for p in prompt_g:
    # Tokenize the prompt
    inputs = tokenizer(p, return_tensors="pt").to("cuda")

    # Generate the response
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_length=200,  # Adjust this based on your needs
            num_beams=5,  # Optional: Use beam search to improve generation quality
            early_stopping=True)

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    out_g.append(response)

In [ ]:
write_out("Baseline_control_general.csv",out_g)


In [ ]:
#run of the prompts 
out_q =[]
for p in prompt_q:
    # Tokenize the prompt
    inputs = tokenizer(p, return_tensors="pt").to("cuda")

    # Generate the response
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_length=200,  # Adjust this based on your needs
            num_beams=5,  # Optional: Use beam search to improve generation quality
            early_stopping=True)

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    out_q.append(response)

One alterantive

In [ ]:
#run of the prompts
count =0
out_q =[]
for p in prompt_q[:100]:
    # Tokenize the prompt
    inputs = tokenizer(p, return_tensors="pt").to("cuda")

    # Generate the response
    outputs = model.generate(
        inputs['input_ids'],
        max_length=200,  # Adjust this based on your needs
        num_beams=2,  # Optional: Use beam search to improve generation quality
        early_stopping=True)

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(count)
    count += 1
    out_q.append(response)

Second Alternative

In [ ]:
out_q =[]
system_prompt = """<s>[INST] <<SYS>>
You are a helpful bot. Your answers are clear and concise.Your answers are in English.
<</SYS>>

"""
for p in prompt_q:
    prompt = system_prompt + f"{p} [/INST]"
    response = get_response(prompt,256)
    response = response[len(prompt):]
    out_q.append(response)

In [ ]:
write_out("/content/gdrive/MyDrive/LLMHallucinations/Baseline_control_qa.csv",out_q)

In [ ]:
#run of the prompts 
out_s =[]
for p in prompt_s:
    # Tokenize the prompt
    inputs = tokenizer("Summarize this document: "+p, return_tensors="pt").to("cuda")

    # Generate the response
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_length=2000,  # Adjust this based on your needs
            num_beams=5,  # Optional: Use beam search to improve generation quality
            early_stopping=True)

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    out_s.append(response)

In [ ]:
write_out("Baseline_control_summarisation.csv",out_s)